# Optimization Problems

The hardest part of solving an optimization problem is often formulating the problem itself.  Optimization problems generally take the form

\begin{align*}
\text{minimize}_x \qquad &f(x)\\
\text{subject to} \qquad &c_e(x) = 0 \\
&c_i(x) \le 0 \\
& l \le x \le u
\end{align*}

where $f(x)$ is some scalar function, $c_e(x)$ is a vector encoding equality constraints, and $c_i(x)$ is a vector encoding inequality constraints.

**Equivalent formulations**
Optimization problems can be represented in many equivalent ways. Typically specific solvers expect a certain form for the optimization problem, so if you're using a solver directly, you may need to transform your problem slightly. Modelling languages like JuMP avoid this by letting you formulate the problem however you want, and then they will transform your formulation into a standard form before passing it into a solver.

For example, if a solver expects problems of the form
\begin{align*}
\text{minimize}_x \qquad &f(x)\\
\text{subject to} \qquad &c_e(x) = 0 \\
& l \le x \le u
\end{align*}
but we have a problem in the form 
\begin{align*}
\text{minimize}_x \qquad &f(x)\\
\text{subject to} \qquad &c_i(x) \leq 0 \\
& l \le x \le u
\end{align*}
we can add additional ``slack'' variables, so that $c_i(x) + s = 0$, and $s \geq 0$, so that the same problem can be written as
\begin{align*}
\text{minimize}_{x,s} \qquad &f(x)\\
\text{subject to} \qquad &c_i(x) + s = 0 \\
& l \le x \le u \\
& 0 \le s
\end{align*}

## Exercise
1. If the solver required that you optimize over $x \geq 0$ only, how would you represent $x \in \mathbb{R}$?
2. If the solver required that you only use equality constraints (so $c(x) = 0$ only) and bound constraints ($l \le x \le u$), how would you represent $l \le c(x) \le u$.

# NEOS
NEOS is a free service originally developed at Argonne National Lab, and now hosted by the University of Wisconsin - Madison. You can submit optimization problems in various standard formats/modelling languages (AMPL, GAMS, CPLEX, MPS, C, Fortran, ...), and choose one of the available solvers (there are tons, including commercial ones!), and they will be solved on their servers (assuming you're allocated enough compute time).

We won't have time to go over many of the modelling languages, but if you are given a problem to solve in one of these formats and you don't have the solvers installed on your machine, you can just submit a job to NEOS.

# Exercise
1. Download pilot4.zip from  http://stanford.edu/group/SOL/multiscale/models/quadLP/MPS/.
2. Go to https://neos-server.org/neos/solvers/index.html and submit the job to any solver which takes MPS input for linear programs.
3. What objective value do you get?

# Numerical Optimization using JuMP


# JuMP

[JuMP](https://github.com/JuliaOpt/JuMP.jl) is a Julia package that provides a modeling language for general optimization problems.

## Optimization in Julia

Julia's optimization packages have become popular due to their ease of use, and variety of interfaces to mature solvers.  The main optimization functionality in Julia is provided by [JuliaOpt](http://www.juliaopt.org/).

The two high-level interfaces that you are most likely to use are
* [JuMP](https://github.com/JuliaOpt/JuMP.jl) - a modeling language for all sorts of optimization problems
* [Convex.jl](https://github.com/JuliaOpt/Convex.jl) - a package for disciplined convex programming (like [CVX](http://cvxr.com/cvx/))

There is a mid-level interface as well, [MathProgBase.jl](https://github.com/JuliaOpt/MathProgBase.jl).

The powerful thing about all of the above is that it is largely **solver independent**. This means you can formulate the optimization problem with these packages, and then choose from a variety of solvers to use under the hood.  This is just like other modeling languages like AMPL - the reason why Julia's optimization packages have become popular is that they are generally easier to use than older modeling languages.

## Solvers

Today is more about turning your optimization models into something that can run on a computer via JuMP, but you still need a solver to actually solve the problem for you under the hood.  [JuliaOpt](http://www.juliaopt.org/) has a list of solvers that can be called from the high level interfaces (there are currently 20).  There are many open-source options available, but there are also interfaces to some of the big commercial solvers such as [Gurobi](http://www.gurobi.com/), [Mosek](https://www.mosek.com/), [Knitro](https://www.artelys.com/en/optimization-tools/knitro), etc. Many of these commercial solvers offer free academic/student licences, and if you are trying to solve large optimization problems they may be worth looking at. Note that these Julia are still being developed, and many solver interfaces may not yet offer the full capabilities of the underlying solver.

In [1]:
# If you have not already installed JuMP and a solver
Pkg.add("JuMP")
Pkg.add("Clp")   # solver
Pkg.add("Ipopt") # solver

INFO: Package JuMP is already installed
INFO: METADATA is out-of-date — you may not have the latest version of JuMP
INFO: Use `Pkg.update()` to get the latest versions of your packages
INFO: Cloning cache of Cbc from https://github.com/JuliaOpt/Cbc.jl.git
INFO: Cloning cache of Clp from https://github.com/JuliaOpt/Clp.jl.git
INFO: Installing Cbc v0.3.2
INFO: Installing Clp v0.4.0
INFO: Building Cbc
INFO: Attempting to create directory /home/restrin/.julia/v0.6/Cbc/deps/downloads
INFO: Downloading file http://www.coin-or.org/download/source/Cbc/Cbc-2.9.8.tgz
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   265  100   265    0     0   1548      0 --:--:-- --:--:-- --:--:--  1778
100 13.0M  100 13.0M    0     0  4041k      0  0:00:03  0:00:03 --:--:-- 4653k
INFO: Done downloading file http://www.coin-or.org/download/source/Cbc/Cbc-2.9.8.tgz
INFO: Attempting to create director

checking build system type... x86_64-unknown-linux-gnu
checking whether we want to compile in debug mode... no
checking for gcc... gcc
checking for C compiler default output file name... a.out
checking whether the C compiler works... yes
checking whether we are cross compiling... no
checking for suffix of executables... 
checking for suffix of object files... o
checking whether we are using the GNU C compiler... yes
checking whether gcc accepts -g... yes
checking for gcc option to accept ANSI C... none needed
configure: C compiler options are: -O3 -pipe -DNDEBUG -pedantic-errors -Wimplicit -Wparentheses -Wsequence-point -Wreturn-type -Wcast-qual -Wall -Wno-unknown-pragmas -Wno-long-long  
checking for g++... g++
checking whether we are using the GNU C++ compiler... yes
checking whether g++ accepts -g... yes
checking whether C++ compiler g++ works... yes
configure: C++ compiler options are: -O3 -pipe -DNDEBUG -pedantic-errors -Wparentheses -Wreturn-type -Wcast-qual -Wall -Wpointer-arith

./configure: line 5844: cd: /home/restrin/.julia/v0.6/Cbc/deps/usr: No such file or directory


/usr/bin/ld
checking if the linker (/usr/bin/ld) is GNU ld... yes
checking for /usr/bin/ld option to reload object files... -r
checking for BSD-compatible nm... /usr/bin/nm -B
checking how to recognise dependent libraries... pass_all
checking how to run the C preprocessor... gcc -E
checking for ANSI C header files... no
checking for sys/types.h... yes
checking for sys/stat.h... yes
checking for stdlib.h... yes
checking for string.h... yes
checking for memory.h... yes
checking for strings.h... yes
checking for inttypes.h... yes
checking for stdint.h... yes
checking for unistd.h... yes
checking dlfcn.h usability... yes
checking dlfcn.h presence... yes
checking for dlfcn.h... yes
checking how to run the C++ preprocessor... g++ -E
checking the maximum length of command line arguments... 32768
checking command to parse /usr/bin/nm -B output from gcc object... ok
checking for objdir... .libs
checking for ar... ar
checking for ranlib... ranlib
checking for strip... strip
checking if gcc suppo

./configure: line 2236: cd: /home/restrin/.julia/v0.6/Cbc/deps/usr: No such file or directory


...

In file included from /home/restrin/.julia/v0.6/Cbc/deps/src/Cbc-2.9.8/Osi/src/Osi/OsiCuts.hpp:14:0,
                 from CbcModel.hpp:13,
                 from CbcCountRowCut.cpp:13:
/home/restrin/.julia/v0.6/Cbc/deps/src/Cbc-2.9.8/Osi/src/Osi/OsiColCut.hpp: In member function 'virtual bool OsiColCut::infeasible(const OsiSolverInterface&) const':
/home/restrin/.julia/v0.6/Cbc/deps/src/Cbc-2.9.8/Osi/src/Osi/OsiColCut.hpp:304:5: warning: this 'if' clause does not guard... [-Wmisleading-indentation]
     if ( cutUbs.isExistingIndex(colIndx) )
     ^~
/home/restrin/.julia/v0.6/Cbc/deps/src/Cbc-2.9.8/Osi/src/Osi/OsiColCut.hpp:306:7: note: ...this statement, but the latter is misleadingly indented as if it were guarded by the 'if'
       if ( newLb > newUb )
       ^~
/home/restrin/.julia/v0.6/Cbc/deps/src/Cbc-2.9.8/Osi/src/Osi/OsiColCut.hpp:315:5: warning: this 'if' clause does not guard... [-Wmisleading-indentation]
     if ( cutLbs.isExistingIndex(colIndx) )
     ^~
/home/restrin/.julia

if /bin/sh ../../libtool --tag=CXX --mode=compile g++ -DHAVE_CONFIG_H -I. -I`echo .` -I../src  -I/home/restrin/.julia/v0.6/Cbc/deps/src/Cbc-2.9.8/Cgl/src -I/home/restrin/.julia/v0.6/Cbc/deps/src/Cbc-2.9.8/Cgl/src/CglAllDifferent -I/home/restrin/.julia/v0.6/Cbc/deps/src/Cbc-2.9.8/Cgl/src/CglClique -I/home/restrin/.julia/v0.6/Cbc/deps/src/Cbc-2.9.8/Cgl/src/CglDuplicateRow -I/home/restrin/.julia/v0.6/Cbc/deps/src/Cbc-2.9.8/Cgl/src/CglFlowCover -I/home/restrin/.julia/v0.6/Cbc/deps/src/Cbc-2.9.8/Cgl/src/CglGMI -I/home/restrin/.julia/v0.6/Cbc/deps/src/Cbc-2.9.8/Cgl/src/CglGomory -I/home/restrin/.julia/v0.6/Cbc/deps/src/Cbc-2.9.8/Cgl/src/CglKnapsackCover -I/home/restrin/.julia/v0.6/Cbc/deps/src/Cbc-2.9.8/Cgl/src/CglLandP -I/home/restrin/.julia/v0.6/Cbc/deps/src/Cbc-2.9.8/Cgl/src/CglLiftAndProject -I/home/restrin/.julia/v0.6/Cbc/deps/src/Cbc-2.9.8/Cgl/src/CglMixedIntegerRounding -I/home/restrin/.julia/v0.6/Cbc/deps/src/Cbc-2.9.8/Cgl/src/CglMixedIntegerRounding2 -I/home/restrin/.julia/v0.6/Cbc/

In file included from /home/restrin/.julia/v0.6/Cbc/deps/src/Cbc-2.9.8/Osi/src/Osi/OsiCuts.hpp:14:0,
                 from CbcModel.hpp:13,
                 from CbcCutGenerator.cpp:25:
/home/restrin/.julia/v0.6/Cbc/deps/src/Cbc-2.9.8/Osi/src/Osi/OsiColCut.hpp: In member function 'virtual bool OsiColCut::infeasible(const OsiSolverInterface&) const':
/home/restrin/.julia/v0.6/Cbc/deps/src/Cbc-2.9.8/Osi/src/Osi/OsiColCut.hpp:304:5: warning: this 'if' clause does not guard... [-Wmisleading-indentation]
     if ( cutUbs.isExistingIndex(colIndx) )
     ^~
/home/restrin/.julia/v0.6/Cbc/deps/src/Cbc-2.9.8/Osi/src/Osi/OsiColCut.hpp:306:7: note: ...this statement, but the latter is misleadingly indented as if it were guarded by the 'if'
       if ( newLb > newUb )
       ^~
/home/restrin/.julia/v0.6/Cbc/deps/src/Cbc-2.9.8/Osi/src/Osi/OsiColCut.hpp:315:5: warning: this 'if' clause does not guard... [-Wmisleading-indentation]
     if ( cutLbs.isExistingIndex(colIndx) )
     ^~
/home/restrin/.juli

if /bin/sh ../../libtool --tag=CXX --mode=compile g++ -DHAVE_CONFIG_H -I. -I`echo .` -I../src  -I/home/restrin/.julia/v0.6/Cbc/deps/src/Cbc-2.9.8/Cgl/src -I/home/restrin/.julia/v0.6/Cbc/deps/src/Cbc-2.9.8/Cgl/src/CglAllDifferent -I/home/restrin/.julia/v0.6/Cbc/deps/src/Cbc-2.9.8/Cgl/src/CglClique -I/home/restrin/.julia/v0.6/Cbc/deps/src/Cbc-2.9.8/Cgl/src/CglDuplicateRow -I/home/restrin/.julia/v0.6/Cbc/deps/src/Cbc-2.9.8/Cgl/src/CglFlowCover -I/home/restrin/.julia/v0.6/Cbc/deps/src/Cbc-2.9.8/Cgl/src/CglGMI -I/home/restrin/.julia/v0.6/Cbc/deps/src/Cbc-2.9.8/Cgl/src/CglGomory -I/home/restrin/.julia/v0.6/Cbc/deps/src/Cbc-2.9.8/Cgl/src/CglKnapsackCover -I/home/restrin/.julia/v0.6/Cbc/deps/src/Cbc-2.9.8/Cgl/src/CglLandP -I/home/restrin/.julia/v0.6/Cbc/deps/src/Cbc-2.9.8/Cgl/src/CglLiftAndProject -I/home/restrin/.julia/v0.6/Cbc/deps/src/Cbc-2.9.8/Cgl/src/CglMixedIntegerRounding -I/home/restrin/.julia/v0.6/Cbc/deps/src/Cbc-2.9.8/Cgl/src/CglMixedIntegerRounding2 -I/home/restrin/.julia/v0.6/Cbc/

Excessive output truncated after 525368 bytes.

In [1]:
using JuMP
using Clp

# Linear Programming

[Linear Programs (LPs)](https://en.wikipedia.org/wiki/Linear_programming) are optimization problems for which both the objective and constraint functions are linear.

\begin{align*}
\text{min or max}_{x\in \mathbb{R}^n} \qquad &c^T x\\
\text{subject to }\qquad &Ax = b \\
&l \le x \le u
\end{align*}

We'll start talking about how to construct models by using LPs as an example.

LPs arise all over the place: maximizing profit, resource allocation, scheduling, max-flow...

We'll explore how to build a model in JuMP by solving a max-flow problem by formulating it as an LP.

**Max Flow Problems**
We have a graph like the one below, and we can think of each edge, $e_i$, as a pipe which has some capacity for carrying water. Our goal is to route as much water as possible from node $s$ to node $t$.

The constraints are the following:
- a pipe carries a non-negative amount of water
- an edge cannot carry more water than its capacity
- the amount of water going into a node is equal to the amount of water exiting the node, except for $s$ and $t$

![alt-text](max_flow_graph.png)

Below, we'll encode this problem into a JuMP model, and then solve it.

In [2]:
## Define problem data for max-flow problem

# Number of nodes and edges
n = 6
m = 8

# Incidence matrix for graph
# B(v,e) = -1 if edge e is exiting from node v
#          +1 if edge e is entering node v

# Thus flow constraint is that sum_{i=1}^m B(i,v) = 0 for all v != s,t
# i.e. sum of across row of B is zero

#        e1 e2 e3 e4 e5 e6 e7 e8
B =     [-1 -1  0  0  0  0  0  0;  #v1
          1  0 -1  0  0  1  0  0;  #v2
          0  1  0 -1 -1  0  0  0;  #v3
          0  0  1  0  1  0 -1  0;  #v4
          0  0  0  1  0 -1  0 -1;  #v5
          0  0  0  0  0  0  1  1]; #v6

# Edge capacities
c = [4 4 5 3 1 2 5 2];

In [3]:
# create a model
mod = Model(solver=ClpSolver())

# add variables (one for each edge)
@variable(mod, 0 <= e[i=1:m] <= c[i])

# Add flow conservation constraint
# Note that we ignore s,t
@constraint(mod, B[2:n-1,:]*e .== 0)

# define objective
@objective(mod, Max, e[1] + e[2])
# prints problem we've defined
mod

Maximization problem with:
 * 4 linear constraints
 * 8 variables
Solver is ClpMathProg

In [4]:
# solve the problem
status = solve(mod)

# print objective value, and the values of x,y
println("Optimal objective: ",getobjectivevalue(mod), 
".\ne = ", getvalue(e))

# status can be one of:
# - :Optimal
# - :Unbounded
# - :Infeasible
# - :UserLimit
# - :Error
# - :NotSolved

Optimal objective: 7.0.
e = [4.0, 3.0, 4.0, 2.0, 1.0, 0.0, 5.0, 2.0]


# Exercise

1. Install another solver that will solve a Linear Program (see [JuliaOpt](http://www.juliaopt.org/) for options - look at the first column), and solve the above example using the new solver.
2. Increase the capacity of edge 7 to be 4. What happens to the optimal flow?
3. Add a constraint so that edge 7 and edge 8 have to have equal flow. What happens now?

# Quadratic Programming

[Quadratic Programs (QPs)](https://en.wikipedia.org/wiki/Quadratic_programming) allow the objective function $f(x)$ to be quadratic.  The optimization problems have the form

\begin{align*}
\text{minimize}_{x\in \mathbb{R}^n} \qquad &\tfrac{1}{2}x^T Q x+c^T x\\
\text{subject to}\qquad &Ax \le b : y \\
& l \le x \le u : z
\end{align*}

Note that in the case that $Q$ is SPD, that the problem is convex, but this is generally not the case. Some solvers (particularly convex ones) will only accept SPD $Q$.

In [33]:
using Ipopt

In [34]:
n = 10
m = 3

Q = rand(n,n)
c = 2*rand(n,1)-1

A = 2*rand(m,n)-1
b = 2*rand(m,1)-1

;

In [35]:
# create a model
# Parameters for the solver can be passed when constructing the solver
# Available parameters depend on the solver
mod2 = Model(solver=IpoptSolver(tol=1e-8))

# add variables
bnds = @variable(mod2, 0 <= x[1:n] <= 1)
# add additional constraint
@constraint(mod2, constr, A*x .<= b)
# define objective
obj = x'*Q*x + c'*x
@objective(mod2, Min, obj[1])

;

In [36]:
solve(mod2)
println("Optimal objective: ",getobjectivevalue(mod2), 
	".\nx = ", getvalue(x))

println("dual vars on bounds = ", getdual(bnds)) # Dual variables z
println("dual vars on constr = ", getdual(constr)) # Dual variables y

This is Ipopt version 3.12.6, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:       30
Number of nonzeros in Lagrangian Hessian.............:      100

Total number of variables............................:       10
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       10
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        3
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        3

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  4.0980056e-03 2.12e-01 8.65e-01  -1.0 0.00e+00    -  0.00e+00 0.00e+00   0
   1

# Recovering primal and dual solutions

JuMP (via MathProgBase) also computes the dual variables to constraints (for example, $y$ and $z$ in the above QP). These can be accessed via ``getdual(c)`` where ``c`` is a reference to the constraint.

Dual variables give information on the sensitivity of the problem to the constraints. You can almost think of it as the derivative of the objective with respect to perturbing the constraints.

For example, if the dual variable to a constraint is 0, then that means that if we perturb the constraint, then the objective value won't change (up to first order).

If the dual variable is large, then changing to constraint a little will change the objective value a lot. For example, it's possible that loosening a constraint with a large dual multiplier a little bit can reduce the objective value substantially.

**Fixed Variables**

Fixed variables are those of the form

``@variable(mod, x==5)``

A couple things to be aware of:
- JuMP still treats these as optimization variables like any others. It will **not** substitute the value.
- This means that the problem class (linear, quadratic, conic, nonlinear...) is determined as if ``x`` is any other variable. So

    ``@constraint(mod, xy = 1)``

    is still a **not** a linear constraint.
- Calling ``setvalue(x, v)`` will cause an error if ``v != 5``. Instead, use ``JuMP.fix(x,v)`` to change the fixed value.
- Treating the fixed variables as regular variables can be useful to learn sensitivity information fromt the dual variable to that constraint. That is, we can learn how the problem depends on that variable's value.

# Modifying Problems

- Swapping solvers: ``setsolver(mod, solverName)``
- changing objective: just call ``@objective`` again
- changing **scalar** variable bounds: ``setlowerbound(x, l)`` and ``setupperbound(x, u)``
- changing **vector** variable bounds: ``setlowerbound.(x, l)`` and ``setupperbound.(x, u)`` (notice the dot)
- adding constraints: just call ``@constraint()`` again
- removing constraints: does not seem currently possible (so build a new model)
- modify constraint right-hand side: ``JuMP.setRHS(constr, v)``

It is possible to add new variables to existing constraints, as long as they appear as linear terms (so of the form ``+a*z`` for some scalar ``a`` and scalar variable ``z``). Otherwise, you would need to rebuild the model.

Once you modify the problem, you'll need to call ``solve()`` again to see the changes.

# Second-order Cone Constraints

http://www.juliaopt.org/JuMP.jl/0.18/refmodel.html#second-order-cone-constraints

These are constraints of the form:
\begin{align*}
\|x_{1:n-1}\| \le x_n,
\end{align*}
or more generally
\begin{align*}
\|Ax-b\|_2 + a^T x + c \le 0.
\end{align*}

JuMP currently only supports conic programs with **linear** objectives.

In [37]:
using ECOS

In [40]:
# Take QP from before and add second order cone constraint
setsolver(mod2, ECOSSolver())

k = 4;
B = 2*rand(k,n)-1
d = rand(k)
a = rand(n)

obj = c'*x
setlowerbound.(x[1:n], -1)
@objective(mod2, Min, obj[1])
@constraint(mod2, norm(B*x - d) + a'*x <= 10)
;

In [41]:
solve(mod2)
println("Optimal objective: ",getobjectivevalue(mod2), 
	".\nx = ", getvalue(x))

Optimal objective: -3.0819559277404283.
x = [-1.0, 1.0, -1.0, -1.0, 1.0, 0.899435, -1.0, 1.0, -0.0487109, -1.0]

ECOS 2.0.5 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +1.729e-01  -2.929e+01  +6e+01  3e-01  6e-01  1e+00  3e+00    ---    ---    1  1  - |  -  - 
 1  -1.402e+00  -5.745e+00  +2e+01  5e-02  2e-01  6e-01  7e-01  0.8109  1e-01   1  1  1 |  0  0
 2  -2.829e+00  -3.433e+00  +2e+00  6e-03  3e-02  9e-02  1e-01  0.8678  2e-02   1  1  1 |  0  0
 3  -2.981e+00  -3.177e+00  +7e-01  2e-03  9e-03  3e-02  3e-02  0.6947  2e-02   1  1  1 |  0  0
 4  -3.075e+00  -3.099e+00  +9e-02  2e-04  1e-03  3e-03  4e-03  0.9040  3e-02   1  1  1 |  0  0
 5  -3.081e+00  -3.083e+00  +8e-03  2e-05  1e-04  3e-04  4e-04  0.9236  2e-02   1  1  1 |  0  0
 6  -3.082e+00  -3.082e+00  +9e-05  2e-07  1e-06  3e-06  4e-06  0.9890  1e-04   1  1  1 |  0  0
 7  -3.082e+00  -3.082e+00  +1

# Semidefinite programming

JuMP supports semidefinite programming as part of its conic programming capabilities. Thus it supports matrices as variables which are either symmetric $X = X^T$ or semidefinite $X \succeq 0$, as well as semidefinite constraints.

Thus we can solve semidefinite programs of the form
\begin{align*}
\min_{X} \qquad & tr(C^T X) \\
\mbox{s.t. } \qquad & tr(A_i^T X) = b_i, \, i=1,\dots, m \\
& X \succeq 0
\end{align*}

To declare semidefinite/symmetric variables, use:

``@variable(mod, X[1:n,1:n], SDP)``
``@variable(mod, X[1:n,1:n], Symmetric)``

and for semidefinite constraints, you use ``@SDconstraint``. For example:
``@SDconstraint(mod, X >= A)``

In [20]:
using SCS

INFO: Precompiling module SCS.


In [32]:
mod4 = Model(solver=SCSSolver())

n = 50
A = rand(n,n)
A = A+A';
@variable(mod4, x>=0)
@objective(mod4, Min, x)
@SDconstraint(mod4, x*eye(n) >= A)

solve(mod4)

println("Optimal objective: ",getobjectivevalue(mod4))

Optimal objective: 108.96314739331747
----------------------------------------------------------------------------
	SCS v1.2.6 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2016
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 2550
eps = 1.00e-04, alpha = 1.80, max_iters = 20000, normalize = 1, scale = 5.00
Variables n = 1275, constraints m = 2550
Cones:	sd vars: 2550, sd blks: 2
Setup time: 5.69e-04s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0|      inf       inf      -nan      -inf       inf       inf  1.79e-03 
    80| 1.06e-05  4.29e-05  1.13e-05  1.09e+02  1.09e+02  0.00e+00  9.91e-02 
----------------------------------------------------------------------------
Status: Solved
Timing: Solve time: 9.9

# Exercise

We'll build a model for doing robust optimization using what we've covered so far.

Suppose there are $n$ stocks, and suppose that the return of the stocks is a Gaussian random vector with mean $\bar{p}$ and covariance $\Sigma$. One simple model for determining what the optimal portfolio is (without short positions) while taking risk into account is the Markowitz model:
\begin{align*}
\max_{x \in \mathbb{R}^n} \qquad & \bar{p}^T x + \frac{\mu}{2} x^T \Sigma x \\
\mbox{subject to} \qquad & \sum_{k=1}^n x_k = 1 \\
& x \geq 0
\end{align*}

Try implementing this model with the given data below. What happens if you try different $\mu$?

Another approach maximizes the expected return while also bounding the probability that the return is less than a certain value. That is, given some $\alpha$ and $\beta < \frac{1}{2}$ it will ensure that $P(\bar{p}^T x \le \alpha) \leq \beta$.

This can be modeled as the problem
\begin{align*}
\max_{x \in \mathbb{R}^n} \qquad & \bar{p}^T \\
\mbox{subject to} \qquad & \bar{p}^T x + \Phi^{-1}(\beta) \|\Sigma^{\frac{1}{2}} x \|_2 \ge \alpha\\
& \sum_{k=1}^n x_k = 1 \\
& x \geq 0
\end{align*}
where $\Phi$ is the CDF of a 1D Gaussian.

Implement the above model, and compare the objective values to the above model.

In [225]:
# Pkg.add("Distributions") #if you don't already have it
using Distributions

INFO: Precompiling module Distributions.


In [226]:
# Required data
n = 10

pbar = 10*rand(n)
Sigma = rand(n,n)
Sigma = Sigma'*Sigma + eye(n)
G = chol(Sigma)
G = Array{Float64, 2}(G)

mu = 0.5
alpha = 4
beta = 0.1

PhiB = quantile.(Normal(), [0, beta])[2] #Φ^{-1}(β)
;